In [1]:
import pandas as pd
import tqdm
import glob
from utils.alphafold import download_alphafold

In [2]:
df = pd.read_csv("./data/main_dataset/main.csv")
df.head(2)

,PDB_wild,uniprot,mutated_chain,mutation_code,pH,Texp,Tm,ddG,dTm,dataset_source,sequence,length,molWeight,countByFeatureType,chain_start,chain_end,AlphaFoldDB,alphafold_path
0,1CQW,P59336,A,V244L,NaN,NaN,52.5,NaN,2.1,FireProtDB,MSEIGTGFPFDPHYVEVLGERMHYVDVGPRDGTPVLFLHGNPTSSY...,294.0,33331.0,"{'Chain': 1, 'Domain': 1, 'Active site': 3, 'B...",1.0,294.0,P59336,./data/main_dataset/3D_structures/alphafold/P5...
1,1CQW,P59336,A,L96V,NaN,NaN,50.0,NaN,-0.4,FireProtDB,MSEIGTGFPFDPHYVEVLGERMHYVDVGPRDGTPVLFLHGNPTSSY...,294.0,33331.0,"{'Chain': 1, 'Domain': 1, 'Active site': 3, 'B...",1.0,294.0,P59336,./data/main_dataset/3D_structures/alphafold/P5...


In [3]:
# download alphafold prediction

for alphafold_id in tqdm.tqdm(df.AlphaFoldDB.unique()):
    download_alphafold(alphafold_id)

100%|██████████| 370/370 [00:00<00:00, 2651.87it/s]

exception raised for alphafold_id: nan:
HTTP Error 404: Not Found


In [4]:
# some entries in the dataset have no linked alphafold id, we try our luck with the uniprot id instead
# already download structures will not be redownloaded (check in download_alphafold)

for uniprot_id in tqdm.tqdm(df.uniprot.unique()):
    download_alphafold(uniprot_id)

  0%|          | 0/430 [00:00<?, ?it/s]

exception raised for alphafold_id: nan:
HTTP Error 404: Not Found


  7%|▋         | 31/430 [00:00<00:02, 164.29it/s]

exception raised for alphafold_id: P69543:
HTTP Error 404: Not Found
exception raised for alphafold_id: P00720:
HTTP Error 404: Not Found
exception raised for alphafold_id: P03034:
HTTP Error 404: Not Found


 11%|█         | 48/430 [00:00<00:03, 96.34it/s] 

...

exception raised for alphafold_id: Q8WZ42:
HTTP Error 404: Not Found
exception raised for alphafold_id: P23074:
HTTP Error 404: Not Found


 75%|███████▌  | 323/430 [00:02<00:00, 122.33it/s]

exception raised for alphafold_id: P69168:
HTTP Error 404: Not Found
exception raised for alphafold_id: P11532:
HTTP Error 404: Not Found
exception raised for alphafold_id: P19551:
HTTP Error 404: Not Found


100%|██████████| 430/430 [00:02<00:00, 182.59it/s]

exception raised for alphafold_id: P04275:
HTTP Error 404: Not Found
exception raised for alphafold_id: P12823:
HTTP Error 404: Not Found


In [5]:
# we now add the path to each record of the dataframe
df["alphafold_path"] = ""

def find_alphafold_path(row):
    alphafold_id = row["AlphaFoldDB"]
    path = glob.glob(f"./data/main_dataset/3D_structures/alphafold/{alphafold_id}.pdb")
    if not path:
        alphafold_id = row["uniprot"]
        path = glob.glob(f"./data/main_dataset/3D_structures/alphafold/{alphafold_id}.pdb")
    
    row["alphafold_path"] = path[0] if path else ""
    return row

df = df.apply(find_alphafold_path, axis=1)
df.to_csv("./data/main_dataset/main.csv", index=False)

In [6]:
df = pd.read_csv("./data/main_dataset/main.csv")
print(f"the current 3D structures folder contains {len(glob.glob('./data/main_dataset/3D_structures/alphafold/*.pdb'))} files")
print(
    f"there are {len(df[pd.isna(df.alphafold_path)])} records without 3D structures path")


the current 3D structures folder contains 408 files
there are 900 records without 3D structures path


In [12]:
from utils.file_utils import write_json
no_3d_struct = df[pd.isna(df.alphafold_path)]
no_3d_struct = no_3d_struct[["PDB_wild", "uniprot", "sequence"]]
no_3d_struct.drop_duplicates(subset=["sequence"], inplace=True)
# put into latch bio template for running it online
# see https://console.latch.bio/workflows/82788/parameters
no_3d_struct.rename(columns={"sequence": "aa_sequence", "uniprot": "run_name"}, inplace=True)
no_3d_struct.drop(columns=["PDB_wild"], inplace=True)
def fasta_format(row):
    row["aa_sequence"] = f"> {row['run_name']}\n"+row["aa_sequence"]
    return row

no_3d_struct = no_3d_struct.apply(fasta_format, axis=1)
no_3d_struct = no_3d_struct[["aa_sequence", "run_name"]]
no_3d_struct.to_csv("./data/main_dataset/3D_structures/no_3d_struct_latchbio.csv", index=False)

# alphafold's shared notebook on google colab:
# https://colab.research.google.com/github/deepmind/alphafold/blob/main/notebooks/AlphaFold.ipynb#scrollTo=woIxeCPygt7K